Сначала сделаем парсер текста. Я взяла за основу код с сайта с корпусом, но с ним выскакивало некоторое количество ошибок, поэтому я его переделала и сделала рабочим. 
Я выбрала именно ревьюшки искусства, потому что корпус с ними сравнительно небольшой, и мне не хотелось слишком сильно загружать им память. 

In [62]:
from gensim.models import KeyedVectors
import gensim.downloader
import gzip
import json
import pandas as pd

In [63]:
def parse(filename):
    f = gzip.open(filename, 'rt')
    entry = {}
    for l in f:
        l = l.strip()
        colonPos = l.find(':')
        if colonPos == -1:
            yield entry
            entry = {}
            continue
        eName = l[:colonPos]
        rest = l[colonPos+2:]
        entry[eName] = rest
    yield entry


arts = []
for e in parse("Arts.txt.gz"):
    arts.append(json.dumps(e))

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Теперь сделаем матрицу пандасом, чтобы понять, какие ключевые слова выделять

In [65]:
arts = [json.loads(i) for i in arts]
matrix = pd.json_normalize(arts)
matrix.sample(50)

,product/productId,product/title,product/price,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text
20551,B00006IF5R,"Scotch Book Tape 845, 2 Inches x 15 Yards",5.27,A1D9V11QUHXENQ,"Big Al ""Alan Rivire""",0/0,5.0,1301616000,It works better than packing tapes,"I own a roll of 2"" Scotch packing tape. I find..."
12613,B0000CBK1L,Brother 1034D 3 or 4 Thread Serger with Easy L...,199.99,A2VEDKMJRSWLQC,Susan Fogel,1/4,1.0,1344297600,Don't Buy the Brother 1034 D serger iti s junk,"Yes, I know that paying $200 or less for a bra..."
18477,B000H89PWY,Royal & Langnickel Large Area Artist Brush Set...,9.99,A29ZSI66WKQM5U,Asmina Jiwa,0/1,3.0,1358467200,It failed expectations,"They are OK. I use the,m but I had to go and g..."
22280,B0002KHBRI,Brother CS-8150 150 Stitch Function Computeriz...,unknown,A16GJIZU6BS2OP,"G. B. Mccormick ""Melissa McCormick""",13/13,5.0,1097107200,Couldn't be happier.,I love this sewing machine!! I would strongly ...
7557,B0009ZHF18,HP Matte Proofing Paper (18 Inches x 100 Feet ...,unknown,A3PKIL09NGWR6Y,J. Beek,0/0,5.0,1193875200,Nice,It's a nice paper. Only issue is paper weight ...
17025,B000B8LEWO,Artists Color Wheel Mixing Guide (Pkg. Of 2),13.95,A1ENU4JYFPOYHO,Klkw1969,5/8,5.0,1179100800,Very helpful,"I bought this for my husband, who is a painter..."
7690,B0002ABB6K,"PM Company Perfection POS/Calculator Rolls, 2....",9.10,A153GDM0CV8W26,Sarge,1/1,5.0,1317427200,great value!!!!,"great value, glad i found it ... saved me mone..."
10985,B00083OPF0,Elastic Foot - For 14SH654 Machine,51.60,A37TWW3M8XRDEY,Sommer Cronck,0/0,4.0,1278720000,Best Price,I searched all over -- both locally and online...
19510,B0007CIOTA,Sullivans Home Hobby Table,126.98,A9FWEEL8AEO15,Jeff Buschow,0/0,4.0,1353542400,OK,It's an OK product and for the price you can't...
13380,B0006HUJ0S,Fiskars 93807097 Circle Cutter with 2 Replacem...,15.30,A1RHHNW75PN24N,Edward W. Robbins,15/18,3.0,1245456000,THIN IS IN,THIS CIRCLE CUTTER WORKS ONLY ON VERY THIN STO...


Тут оказалось много разных ключевых слов типа Set, Brother, Kit, Sewing, Color, Tool, Paper, Pen. 
В принципе, в области искусства действительно большое разнообразие товаров - от картин до вышивки и инструментов. 
Если так обобщать, то, мне кажется, тут можно выделить названия типов "произведений" искусства и инструментов - таких как sewing, picture, tool, kit. 
Самым простым и логичным будет сделать list со всеми ключевыми словами, которые можно кратенько встретить в заголовках. Конечно, это будет не полный список, потому что в таком большом корпусе сложно учесть прямо все. Чтобы расширить возможное семантическое поле, я воспользуюсь word2vec и посмотрю, какие ближайшие соседи могут быть у слов, которые я накидала. 
В качестве корпуса использую предобученный корпус из gensim. 

В качестве ориентира я использовала код из вот этого туториала:https://github.com/zhlli1/Genism-word2vec/blob/master/Genism%20Word2Vec%20Tutorial.ipynb

In [66]:
from gensim.models import KeyedVectors
filename = 'fasttext-wiki-news-subwords-300'
model = gensim.downloader.load(filename)

INFO - 08:47:23: loading projection weights from /root/gensim-data/fasttext-wiki-news-subwords-300/fasttext-wiki-news-subwords-300.gz
INFO - 08:53:17: loaded (999999, 300) matrix from /root/gensim-data/fasttext-wiki-news-subwords-300/fasttext-wiki-news-subwords-300.gz


In [67]:
keywords = ['tool', 'kit', 'sewing', 'art', 'picture', 'present',
            'pen',
            'color', 'colors', 'canvas', 'tape', 'wool', 'paint', 'thread',  
            'cutting', 'dyes', 'photo', 'tube', 'marker', 'pastel', 'oil',
            'glue', 'paper', 'jewerly']


In [68]:
names = {}

for i in keywords:
  names[i] = model.most_similar(i)

namelist = []
namelist = list(functools.reduce(lambda x, y: x + y, names.items()))
print(namelist)

INFO - 08:53:17: precomputing L2-norms of word weight vectors


['tool', [('tools', 0.7866291999816895), ('tool-kit', 0.7492363452911377), ('tool-', 0.7362332344055176), ('toolbox', 0.7358964681625366), ('tools-', 0.7269669771194458), ('toolset', 0.7231194376945496), ('tool-set', 0.7168781757354736), ('toolkit', 0.7168464064598083), ('multi-tool', 0.7157315015792847), ('tool-box', 0.7052396535873413)], 'kit', [('kits', 0.8452795743942261), ('kit.', 0.690009593963623), ('kitbag', 0.6788609027862549), ('tool-kit', 0.6091724038124084), ('caboodle', 0.5982969403266907), ('toolkit', 0.591160774230957), ('gear', 0.58803391456604), ('drum-kit', 0.5869807004928589), ('press-kit', 0.5787755846977234), ('kaboodle', 0.5786168575286865)], 'sewing', [('knitting', 0.7823529243469238), ('embroidery', 0.7060049772262573), ('dressmaking', 0.7017966508865356), ('sewing-machine', 0.6957395672798157), ('crocheting', 0.6852182745933533), ('stitching', 0.6834543943405151), ('quilting', 0.6799575090408325), ('sew', 0.6758464574813843), ('needlework', 0.6750808954238892),

Теперь у нас есть список со всеми ближайшими словами для нашего словаря. Можно еще расширять список keywords полученными из word2vec словами и получать еще больше возможных соседей.Вовсе не обязательно потом переводить словарь в список, но мне так захотелось.

Другие методы:
1. Залезть в категории на Амазоне, посмотреть, что пишут там, и по этим данным составить список возможных упоминаний. Это вообще касается любого крупного маркетплейса или той же Википедии. Можно вообще из нескольких источников набрать. Это довольно широкая область, но придется искать и выписывать все руками, либо писать отдельный парсер.
2. Найти топ самых частотных слов в корпусе ревью или среди самих заголовков. Там, правда, не факт, что самыми частотными словами будут именно нужные нам упоминания товаров. Это решается через стоп-слова в нлтк, но, как мне кажется, там может быть и много прилагательных, и такие вещи как author, artsy piece, thing. То есть более абстрактные обозначения товаров. Мне кажется, это не очень продуктивный подход.